In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from requests_oauthlib import OAuth1Session
import string
import sqlite3
import time
import datetime

In [ ]:
from itertools import chain, product
def bruteforce(charset, maxlength):
    return (''.join(candidate)
        for candidate in chain.from_iterable(product(charset, repeat=i)
            for i in range(1, maxlength + 1)))

In [ ]:
def insert_user(user, cur):
    cur.execute('INSERT OR REPLACE INTO accounts VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)',(
        user['id'], user['screen_name'], user['name'],user['created_at'],
        user['description'], user['followers_count'], user['friends_count'],
        user['listed_count'], user['statuses_count'],
        user['profile_image_url'], user['profile_background_image_url'],
        user['time_zone'], user['protected']))

In [ ]:
CK = ''
CS = ''
AT = ''
AS = ''

In [ ]:
twitter = OAuth1Session(CK, CS, AT, AS)

In [ ]:
conn = sqlite3.connect('twitter_accounts.db')
cur = conn.cursor()
cur.execute('''
    CREATE TABLE IF NOT EXISTS accounts(
        id integer primary key,
        screen_name text,
        name text,
        created_at text,
        description text,
        followers_count integer,
        friends_count integer,
        listed_count integer,
        statuses_count integer,
        profile_image_url text,
        profile_background_image_url text,
        time_zone text,
        protected integer
    )''')
cur.execute('''
    CREATE TABLE done_screen_names(
    screen_name text)''')

In [ ]:
cnt = 0
hit = 0
url = 'https://api.twitter.com/1.1/users/show.json?screen_name={}&include_entities=true'
done_screen_names = [row[0] for row in cur.execute('SELECT screen_name from done_screen_names').fetchall()]
for attempt in bruteforce(string.ascii_lowercase, 3):
    cnt += 1
    screen_name = 's{}_okuma'.format(attempt)
    cur.execute('INSERT INTO done_screen_names VALUES(?)', (screen_name,))
    if screen_name in done_screen_names:
        continue
    user_json = twitter.get(url.format(screen_name)).json()
    print('{}/{}: {}'.format(hit, cnt, screen_name), end='\r')
    if 'errors' in user_json:
        if 'limit' in user_json['errors'][0]['message']:
            reset = twitter.get('https://api.twitter.com/1.1/application/rate_limit_status.json').json()['resources']['users']['/users/show/:id']['reset']
            print('reset at ', datetime.datetime.fromtimestamp(reset))
            time.sleep(60 + reset - int(time.time()))
            user_json = twitter.get(url.format(screen_name)).json()
            if 'errors' in user_json:
                continue
        else:
            print('{}/{}, {}'.format(hit, cnt, screen_name), end='\r')
            continue
    hit += 1
    insert_user(user_json, cur)
    conn.commit()
    print('{}/{}. {}: {}'.format(hit, cnt, user_json['screen_name'], user_json['description']))
    time.sleep(2)

In [ ]:
cur.close()